# CONFLUENCE Tutorial - 6: Regional Domain Modeling (Iceland)

## Introduction

This tutorial marks a fundamental shift in scale and approach: from single-watershed modeling to regional domain modeling. Instead of focusing on one watershed draining to a single outlet, we now model an entire region containing multiple independent drainage systems. Using Iceland as our example, we'll demonstrate how CONFLUENCE handles complex regional hydrology including coastal watersheds, multiple drainage basins, and diverse hydrological regimes within a single modeling framework.

### What is Regional Domain Modeling?

Regional domain modeling treats an entire geographic region (country, province, or large area) as the modeling domain:

- **Multiple watersheds**: The domain contains many independent drainage systems
- **Coastal watersheds**: Basins that drain directly to the ocean, not to interior points
- **Regional water resources**: Understanding water availability across an entire region
- **Diverse conditions**: Multiple climate zones, topographic settings, and hydrological regimes

### Key Conceptual Differences

| Aspect | Watershed Modeling | Regional Modeling |
|--------|-------------------|------------------|
| **Drainage** | Single outlet point | Multiple independent outlets |
| **Boundaries** | Topographic divides | Administrative or geographic boundaries |
| **Scale** | 10s - 1000s km² | 10,000s - 100,000s km² |
| **Outlets** | Rivers/streams | Rivers and coastal discharge |
| **Purpose** | Basin-specific studies | Regional water resources |

### Why Iceland?

Iceland provides an excellent case study for regional modeling because:

**Geographic characteristics**:
- **Island nation**: Clear regional boundaries with diverse drainage systems
- **Volcanic landscape**: Complex topography creating numerous independent watersheds
- **Coastal orientation**: Many watersheds drain directly to the ocean
- **Size**: Manageable for demonstration (~103,000 km²) but large enough to show regional concepts

**Hydrological diversity**:
- **Glacial systems**: Ice-dominated watersheds with unique hydrology
- **Volcanic terrain**: Permeable basalts affecting groundwater flow
- **Coastal climate**: Maritime climate with strong seasonal variations
- **Elevation gradients**: From sea level to >2000m elevation

**Practical advantages**:
- **Data availability**: Good coverage of meteorological and topographic data
- **Computational feasibility**: Large enough to demonstrate concepts, small enough for tutorial execution
- **Clear boundaries**: Island setting eliminates boundary condition complexities

### Technical Implementation

Regional modeling requires several key technical approaches:

1. **Bounding box delineation**: Define the region using geographic coordinates rather than pour points
2. **Coastal watershed inclusion**: Capture watersheds that drain directly to ocean boundaries
3. **Multiple drainage systems**: Handle independent watersheds without upstream-downstream connections
4. **Regional forcing**: Apply meteorological data across diverse topographic and climate conditions

### Scientific Applications

Regional domain modeling is valuable for:

- **National water resources**: Understanding water availability across entire countries
- **Climate change impacts**: Assessing regional-scale climate change effects on hydrology
- **Comparative hydrology**: Studying how different watersheds respond to similar forcing
- **Policy applications**: Supporting water management decisions at regional scales
- **Ecosystem services**: Quantifying water-related ecosystem services across regions

### Key Configuration Changes

For regional modeling, several configuration parameters change:

**Domain setup**:
- **DELINEATE_BY_POURPOINT**: `False` (use bounding box instead)
- **BOUNDING_BOX_COORDS**: Regional extent rather than watershed-specific
- **DELINEATE_COASTAL_WATERSHEDS**: `True` (include ocean-draining basins)

**Computational considerations**:
- **STREAM_THRESHOLD**: Higher values to manage complexity
- **Spatial resolution**: Balance between detail and computational feasibility
- **Processing time**: Longer execution due to regional scale

### Challenges and Considerations

Regional modeling introduces several challenges:

1. **Computational complexity**: Much larger domains require more processing time and memory
2. **Data heterogeneity**: Diverse conditions may require different parameterizations
3. **Boundary conditions**: Coastal boundaries introduce ocean interaction effects
4. **Validation**: Fewer gauging stations relative to the number of watersheds
5. **Interpretation**: More complex results requiring regional-scale analysis approaches

### What You'll Learn

This tutorial will teach you how to:

1. **Configure regional domains** using bounding boxes rather than pour points
2. **Handle coastal watersheds** that drain directly to ocean boundaries
3. **Manage multiple drainage systems** within a single modeling framework
4. **Apply regional-scale forcing** data across diverse topographic conditions
5. **Interpret regional results** including multiple independent watersheds
6. **Understand computational trade-offs** at regional scales

### Tutorial Structure

This tutorial follows the familiar CONFLUENCE workflow but with regional-scale considerations:

1. **Regional Setup**: Configure domain using bounding box coordinates
2. **Multi-watershed Delineation**: Identify all watersheds within the regional domain
3. **Coastal Integration**: Include watersheds that drain to ocean boundaries
4. **Regional Data Processing**: Apply forcing data across the entire region
5. **Model Execution**: Run the regional model with multiple drainage systems
6. **Regional Analysis**: Interpret results across multiple watersheds and drainage systems

### Expected Outcomes

By completing this tutorial, you'll understand:
- How to scale hydrological modeling from watersheds to regions
- The technical requirements for regional domain modeling
- How to handle multiple independent drainage systems
- The scientific applications of regional-scale hydrology
- The computational and data management challenges of large-scale modeling

This tutorial represents the largest spatial scale in our series and demonstrates CONFLUENCE's capability to handle complex regional hydrology while maintaining the same workflow efficiency that we've used for smaller-scale applications.

## 1. Setup

In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path
import yaml
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
from shapely.geometry import box
import contextily as cx
from datetime import datetime
import xarray as xr
import warnings

# Suppress specific warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

# Add CONFLUENCE to path
confluence_path = Path('../').resolve()
sys.path.append(str(confluence_path))

# Import CONFLUENCE
from CONFLUENCE import CONFLUENCE

plt.style.use('default')
%matplotlib inline

## 2. Initialize CONFLUENCE
First, let's set up our directories and load the configuration. We'll customize the Iceland configuration file.

In [ ]:
# Set directory paths
CONFLUENCE_CODE_DIR = confluence_path
CONFLUENCE_DATA_DIR = Path('/work/comphyd_lab/data/CONFLUENCE_data')  # ← User should modify this path

# Load Iceland configuration
iceland_config_path = CONFLUENCE_CODE_DIR / '0_config_files' / 'config_Iceland.yaml'
with open(iceland_config_path, 'r') as f:
    config_dict = yaml.safe_load(f)

# Create tutorial version with updated domain name
config_dict['DOMAIN_NAME'] = "Iceland_tutorial"  
config_dict['EXPERIMENT_ID'] = "tutorial_run_1"
config_dict['EXPERIMENT_TIME_START'] = '2011-01-01 01:00'
config_dict['EXPERIMENT_TIME_END'] = '2022-12-31 23:00'

# Create config directory if it doesn't exist
config_dir = CONFLUENCE_CODE_DIR / '0_config_files'
config_dir.mkdir(parents=True, exist_ok=True)

# Write to tutorial config file
tutorial_config_path = config_dir / 'config_iceland_tutorial.yaml'
with open(tutorial_config_path, 'w') as f:
    yaml.dump(config_dict, f)

# Initialize CONFLUENCE with tutorial config
confluence = CONFLUENCE(tutorial_config_path)

# Parse bounding box for visualization
bbox = config_dict['BOUNDING_BOX_COORDS'].split('/')
lat_max, lon_min, lat_min, lon_max = map(float, bbox)

# Display configuration
print("=== Iceland Tutorial Configuration ===")
print(f"Domain Name: {confluence.config['DOMAIN_NAME']}")
print(f"Bounding Box: {confluence.config['BOUNDING_BOX_COORDS']}")
print(f"Delineate by Pour Point: {confluence.config['DELINEATE_BY_POURPOINT']} (Full region!)")
print(f"Include Coastal Watersheds: {confluence.config.get('DELINEATE_COASTAL_WATERSHEDS', True)}")
print(f"Stream Threshold: {confluence.config['STREAM_THRESHOLD']}")
print(f"Domain Method: {confluence.config['DOMAIN_DEFINITION_METHOD']}")

## 4. Project Setup
The first step is to set up our project structure. Since we're doing a regional model, we need a different approach than for a single watershed.

In [ ]:
project_dir = confluence.managers['project'].setup_project()    
pour_point_path = confluence.managers['project'].create_pour_point()

# List created directories
print("\nCreated directories:")
created_dirs = []
for item in sorted(project_dir.iterdir()):
    if item.is_dir():
        created_dirs.append(item.name)
        print(f"  📁 {item.name}")

# Check if all required directories are created
required_dirs = ['shapefiles', 'attributes', 'forcing', 'simulations', 'evaluation', 'plots']
for dir_name in required_dirs:
    if dir_name not in created_dirs:
        print(f"Warning: Required directory '{dir_name}' not created")

print("\nDirectory purposes:")
print("  shapefiles: Domain geometry (multiple watersheds, river network)")
print("  attributes: Static characteristics (elevation, soil, land cover)")
print("  forcing: Meteorological inputs (precipitation, temperature)")
print("  simulations: Model outputs")
print("  evaluation: Performance metrics and comparisons")
print("  plots: Visualizations")


## 5. Geospatial Domain Definition and Analysis - Data Acquisition
Before delineating the region, we need to acquire geospatial data (DEM, soil, land cover).

In [ ]:
print("Acquiring geospatial attributes (DEM, soil, land cover)...")
confluence.managers['data'].acquire_attributes()

## 6. Regional Domain Delineation
This is the critical step where we delineate the entire region, including coastal watersheds. This is different from the single-watershed approach.

In [ ]:
print(f"Delineating regional domain using method: {confluence.config['DOMAIN_DEFINITION_METHOD']}")
print(f"Delineate by pour point: {confluence.config['DELINEATE_BY_POURPOINT']} (Full region!)")
print(f"Include coastal watersheds: {confluence.config.get('DELINEATE_COASTAL_WATERSHEDS', True)}")
print(f"Stream threshold: {confluence.config['STREAM_THRESHOLD']}")
print("\nThis will create multiple independent drainage basins...")

watershed_path = confluence.managers['domain'].define_domain()

# Check results
basin_path = project_dir / 'shapefiles' / 'river_basins'
network_path = project_dir / 'shapefiles' / 'river_network'

basin_count = 0
basin_files = []
basins = None
if basin_path.exists():
    basin_files = list(basin_path.glob('*.shp'))
    if basin_files:
        try:
            basins = gpd.read_file(basin_files[0])
            basin_count = len(basins)
            print(f"\n✓ Created {basin_count} watersheds")
            print(f"Total area: {basins.geometry.area.sum() / 1e6:.0f} km²")
        except Exception as e:
            print(f"Error reading basin shapefile: {str(e)}")

network_count = 0
network_files = []
rivers = None
if network_path.exists():
    network_files = list(network_path.glob('*.shp'))
    if network_files:
        try:
            rivers = gpd.read_file(network_files[0])
            network_count = len(rivers)
            print(f"✓ Created river network with {network_count} segments")
        except Exception as e:
            print(f"Error reading river network shapefile: {str(e)}")
            
if not basin_files:
    print("⚠ No basin shapefiles found. Domain delineation may have failed.")
if not network_files:
    print("⚠ No river network shapefiles found. Stream delineation may have failed.")

## 7. Watershed Discretization
Now we need to discretize our domain into GRUs (Grouped Response Units) and HRUs (Hydrologic Response Units).

In [ ]:
print(f"Creating HRUs using method: {confluence.config['DOMAIN_DISCRETIZATION']}")
hru_path = confluence.managers['domain'].discretize_domain()

## 8. Visualize Regional Domain
Let's visualize what our regional domain looks like with all delineated watersheds.

In [ ]:
# Create CONFLUENCE domain visualization
print("Creating regional domain visualization...")
if hasattr(confluence.managers['domain'], 'plot_domain'):
    plot_paths = confluence.managers['domain'].plot_domain()
else:
    print("plot_domain method not available - using custom visualization instead")

# Create custom visualization
if basin_path.exists() and basin_files and basins is not None:
    fig, ax = plt.subplots(figsize=(14, 10))
    
    # Plot watersheds
    if 'GRU_ID' in basins.columns:
        basins.plot(ax=ax, column='GRU_ID', cmap='tab20', 
                   edgecolor='black', linewidth=0.5, legend=False)
    else:
        basins.plot(ax=ax, cmap='tab20', 
                   edgecolor='black', linewidth=0.5, legend=False)
    
    # Plot river network if available
    if network_path.exists() and network_files and rivers is not None:
        rivers.plot(ax=ax, color='blue', linewidth=1)
    
    ax.set_title(f'Iceland Regional Domain - {basin_count} Watersheds', 
                fontsize=16, fontweight='bold')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    
    # Add annotation about coastal watersheds
    ax.text(0.02, 0.98, f'Including coastal watersheds\nTotal watersheds: {basin_count}',
            transform=ax.transAxes, va='top',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.8),
            fontsize=12)
    
    plt.tight_layout()
    plt.show()
else:
    print("Cannot create visualization: Basin data not available")

## Acquire forcing data

In [ ]:
print(f"\nAcquiring forcing data: {confluence.config['FORCING_DATASET']}")
confluence.managers['data'].acquire_forcings()

## Model Agnostic Preprocessing 

In [ ]:
print("\nRunning model-agnostic preprocessing...")
confluence.managers['data'].run_model_agnostic_preprocessing()

## Model Specific Preprocessing

In [ ]:
print(f"Preparing {confluence.config['HYDROLOGICAL_MODEL']} input files...")
confluence.managers['model'].preprocess_models()

## Model initiation

In [ ]:
print(f"\nRunning {confluence.config['HYDROLOGICAL_MODEL']} model...")
confluence.managers['model'].run_models()

print("\nModel run complete")